<table style="width: 100%;"> <tr> <td style="width: 20%; vertical-align: top;"> <img src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" alt="UdeA" height="150px"> </td> <td style="width: 80%; padding-left: 20px;"> <strong style="font-size: 30px;">MODELOS DE RECOMENDACIÓN PARA EL DESARROLLO DE UN SISTEMA DE RECOMENDACIÓN DE PELÍCULAS - PROYECTO MARKETING</strong><br><br> <span style="font-size: 30px;"> Luisa Fernanda Alzate Cuartas <br> Juan Camilo Henao Caro<br>Isabella Mendez Hoyos<br> Fernando Antonio Piñeres Ramos </span> </td> </tr> </table>

#### LIBRERÍAS REQUERIDAS

In [ ]:
#Conexión de Google Colab con Drive.
from google.colab import drive
drive.mount('/content/drive')

#Acceder a los archivos de la carpeta marketing para el uso de funciones.
import sys
sys.path.append('/content/drive/MyDrive/marketing/Marketing')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd # Manipulación y el analísis de datos.
import sqlite3 as sql # Interactuar con base de datos SQL usando Python.
from sklearn.metrics.pairwise import cosine_similarity # Calcular similitudes entre vectores.
from sklearn.preprocessing import normalize
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

#### CONEXIÓN CON LA BASE DE DATOS

In [ ]:
#Establece una conexión a la base de datos SQLite ubicada en "content/db_movies"
conexion = sql.connect('/content/drive/MyDrive/marketing/Marketing/data/db_movies_copy.db')

# Crear un cursor a partir de la conexión, que permita ejecutar comando SQL sobre la base de datos.
cur = conexion.cursor()

In [ ]:
# Ejecuta una consulta SQL que obtiene los nombres de todas las tablas existentes en la base de datos.
cur.execute("SELECT name FROM sqlite_master where type='table' ")

# Recupera todos los resultados de la consulta anterior en forma de lista de tuplas.
cur.fetchall()

[('ratings',), ('movies',), ('ratings_movies',)]

In [ ]:
table_movies = pd.read_sql_query("SELECT * FROM movies", conexion)
table_ratings = pd.read_sql_query("SELECT * FROM ratings", conexion)

#### **SISTEMAS DE RECOMENDACIÓN PARA UNA PLATAFORMA DE PELÍCULAS EN LÍNEA.**
-------

#### POPULARIDAD

##### PELÍCULAS MÁS POPULARES - MAYOR CANTIDAD DE CALIFICACIONES.



```
# Se utilizaron consultas SQL para identificar las películas con mayor número de calificaciones y mejor puntuación promedio.
Se aplicaron filtros para asegurar un mínimo de califiaciones por película y se representaron los resultados.
```



In [ ]:
#Sistema de recomendanción basado en popularidad: Las 6 películas con mayor cantidad de calificaciones.

query = """
SELECT
    m.titulo AS titulo_pelicula,
    COUNT(*) AS total_visualizaciones

FROM ratings r
JOIN movies m ON r.movieId = m.movieId
GROUP BY m.movieId
ORDER BY total_visualizaciones DESC
LIMIT 6;
"""

peliculas_mas_populares = pd.read_sql(query, conexion)
display(peliculas_mas_populares)

,titulo_pelicula,total_visualizaciones
0,Forrest Gump,329
1,"Shawshank Redemption, The",317
2,Pulp Fiction,307
3,"Silence of the Lambs, The",279
4,"Matrix, The",278
5,Star Wars: Episode IV - A New Hope,251


##### PELÍCULAS MÁS POPULARES POR DÍA.

In [ ]:
# Sistema de recomendación por pupularidad: Películas más popularen por día, basado en el pronedio de la calificación.
dia = 2 #Se puede cambiar el número del día.

query = f"""
SELECT r.titulo, r.day, AVG(r.rating) AS avg_rating
FROM ratings_movies r
WHERE r.day = {dia}
GROUP BY r.day, r.titulo
ORDER BY r.day, avg_rating DESC
    LIMIT 6;
"""
pd.read_sql(query, conexion)

,titulo,day,avg_rating
0,You Only Live Twice,2,5.0
1,World of Tomorrow,2,5.0
2,Winnie the Pooh and the Blustery Day,2,5.0
3,What Lies Beneath,2,5.0
4,Wayne's World 2,2,5.0
5,Wallace & Gromit: The Wrong Trousers,2,5.0


##### PELÍCULAS MÁS POPULARES POR MES.

In [ ]:
# Sistema de recomendación por pupularidad: Películas más populares en el mes, basado en el pronedio de la calificación.

mes = 3 #Se puede cambiar el número del mes.

query = f"""
SELECT r.titulo, r.month, AVG(r.rating) AS avg_rating
FROM ratings_movies r
WHERE r.month = {mes}
GROUP BY r.month, r.titulo
ORDER BY r.month, avg_rating DESC
    LIMIT 6;
"""
pd.read_sql(query, conexion)

,titulo,month,avg_rating
0,eXistenZ,3,5.0
1,Zulu,3,5.0
2,Zero Dark Thirty,3,5.0
3,"Zed & Two Noughts, A",3,5.0
4,You Can Count on Me,3,5.0
5,Yi Yi,3,5.0


##### POPULARIDAD DE PELICULAS MÁS RECIENTES - MAYOR CANTIDAD DE CALIFICACIONES.

In [ ]:
#Sistemas de recomendación por popularidad: Peliculas recientes más populares-basadas en la cantidad de calificaciones.

year = 2010 # Se puede cambiar el número del año.

query = f"""SELECT m.titulo, COUNT(*) AS num_ratings
FROM ratings r
JOIN movies m ON r.movieId = m.movieId
WHERE r.rating > 0
GROUP BY m.movieId
HAVING año > {year}
ORDER BY num_ratings DESC
LIMIT 6;
"""
pd.read_sql_query(query, conexion)

,titulo,num_ratings
0,"Dark Knight Rises, The",76
1,Interstellar,73
2,Django Unchained,71
3,"Avengers, The",69
4,Guardians of the Galaxy,59
5,Deadpool,54


#### SISTEMA DE RECOMENDACIÓN BASADA EN CONTENIDO O SIMILITUD.
--------



```
# Se creó un sistema de recomendación basado en el contenido de las películas.

 Utilizando los géneros de cada película (representados de forma binaria) y la librería sklearn de Python,
 se calculó una matriz de similitud entre las películas empleando la métrica del coseno.

 Esto permite medir qué tan parecidas son las películas según sus géneros y,
 de esta manera, ofrecer recomendaciones más relevantes para el usuario.
```



In [ ]:
#Se crea un query que almacena los registros que almacena la tabla movies y evitar borrar información del orginal.
query_generos = """
SELECT movieId, titulo,
       Action, Comedy, Drama, Horror, Romance, Thriller,
       Adventure, Animation, Children, Documentary,
       Fantasy, IMAX, Musical, Mystery, War, Western, "Film-Noir"
FROM movies;
"""

peliculas_df = pd.read_sql_query(query_generos, conexion)
peliculas_df.set_index('movieId', inplace=True) # Elimina el indica que al tabla y lo traslada a la columna movieId.

In [ ]:
# Extraer solo columnas de géneros
generos = peliculas_df.drop(columns=['titulo'])

# Calcular matriz de similitud
similitud = cosine_similarity(generos)

# Convertir a DataFrame
similitud_df = pd.DataFrame(similitud, index=peliculas_df.index, columns=peliculas_df.index)

In [ ]:
#tabla que muestra qué tan similares son todas las películas entre sí según sus géneros, utilizando la similitud coseno como medida.
similitud_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.774597,0.316228,0.258199,0.447214,0.0,0.316228,0.632456,0.0,0.258199,...,0.516398,0.316228,0.316228,0.447214,0.0,0.670820,0.774597,0.00000,0.316228,0.447214
2,0.774597,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.816497,0.0,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.0,0.288675,0.333333,0.00000,0.000000,0.000000
3,0.316228,0.000000,1.000000,0.816497,0.707107,0.0,1.000000,0.000000,0.0,0.000000,...,0.408248,0.000000,0.500000,0.000000,0.0,0.353553,0.408248,0.00000,0.000000,0.707107
4,0.258199,0.000000,0.816497,1.000000,0.577350,0.0,0.816497,0.000000,0.0,0.000000,...,0.333333,0.408248,0.816497,0.000000,0.0,0.288675,0.333333,0.57735,0.000000,0.577350
5,0.447214,0.000000,0.707107,0.577350,1.000000,0.0,0.707107,0.000000,0.0,0.000000,...,0.577350,0.000000,0.707107,0.000000,0.0,0.500000,0.577350,0.00000,0.000000,1.000000


In [ ]:
#Función del sistema de recomendación basado en contenido: Al colocar el nombre de la película recomienda aquellas que sean similares.

def recomendar_peliculas(movie_title, peliculas_df, similitud_df, top_n=4):
    # Buscar el ID de la película
    pelicula_id = peliculas_df[peliculas_df['titulo'].str.lower() == movie_title.lower()].index

    if pelicula_id.empty:
        print("Película no encontrada.")
        return

    pelicula_id = pelicula_id[0]

    # Obtener las similitudes
    similares = similitud_df[pelicula_id].sort_values(ascending=False)

    # Excluir la misma película
    similares = similares.drop(pelicula_id)

    # Obtener títulos de las películas similares
    recomendadas = peliculas_df.loc[similares.head(top_n).index, 'titulo']

    print(f"\n🎬 Películas similares a: {peliculas_df.loc[pelicula_id, 'titulo']}\n")
    for i, titulo in enumerate(recomendadas, 1):
        print(f"{i}. {titulo}")

In [ ]:
# Ejemplo utilizando la películas Forrest Gump
recomendar_peliculas("Forrest Gump", peliculas_df, similitud_df)


🎬 Películas similares a: Forrest Gump

1. Life Is Beautiful (La Vita è bella)
2. I Served the King of England (Obsluhoval jsem anglického krále)
3. Train of Life (Train de vie)
4. Tiger and the Snow, The (La tigre e la neve)


#### SISTEMA DE RECOMENDACIÓN COLABORATIVA.
-------



```
# Se desarrolló un sistema de recomendación de filtrado colaborativo que encuentra usuarios con patrones de calificación similares.

Para esto, se creó una matriz usuario-película, se imputaron las calificaciones
faltantes con la media de cada usuario, y se calculó la similitud coseno entre
usuarios. Finalmente, se recomiendan al usuario actual películas que usuarios
similares han visto y calificado positivamente, basándose en el comportamiento colectivo.
```



In [ ]:
# Crear la matriz de usuario vs película
ratings_matrix = table_ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Ver la matriz de calificaciones
ratings_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Rellenar valores nulos con la media de las calificaciones de cada película
ratings_matrix_filled = ratings_matrix.apply(lambda row: row.fillna(row.mean()), axis=1)

# Ver la matriz con valores completados
ratings_matrix_filled.head()


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,4.366379,4.000000,4.366379,4.366379,4.000000,4.366379,4.366379,4.366379,4.366379,...,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379
2,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,...,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276
3,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,...,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897
4,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,...,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556
5,4.000000,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,...,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364


In [ ]:
# Calcular la similitud entre usuarios
user_similarity = cosine_similarity(ratings_matrix_filled)

# Convertir la matriz de similitudes a un DataFrame
user_similarity_df = pd.DataFrame(user_similarity, index=ratings_matrix_filled.index, columns=ratings_matrix_filled.index)

# Ver las primeras similitudes
user_similarity_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.999540,0.998159,0.998163,0.999446,0.998587,0.998273,0.999449,0.999261,0.998668,...,0.999552,0.999009,0.994340,0.999343,0.998831,0.997377,0.999031,0.994773,0.999559,0.996018
2,0.999540,1.000000,0.998498,0.998417,0.999780,0.998973,0.998616,0.999754,0.999586,0.999020,...,0.999880,0.999354,0.994720,0.999706,0.999202,0.997698,0.999308,0.994891,0.999899,0.996354
3,0.998159,0.998498,1.000000,0.997014,0.998363,0.997614,0.997242,0.998350,0.998205,0.997638,...,0.998501,0.997955,0.993444,0.998324,0.997818,0.996180,0.997920,0.993453,0.998523,0.995043
4,0.998163,0.998417,0.997014,1.000000,0.998293,0.997565,0.997335,0.998314,0.998125,0.997689,...,0.998411,0.998031,0.993432,0.998237,0.997835,0.996319,0.997892,0.993245,0.998444,0.994878
5,0.999446,0.999780,0.998363,0.998293,1.000000,0.998886,0.998527,0.999656,0.999482,0.998888,...,0.999785,0.999269,0.994668,0.999606,0.999072,0.997621,0.999232,0.994790,0.999814,0.996226


In [ ]:
#Función del sistema de recomendación colaborativa: Basado en lo que han visto otros usuarios y le puede interesar al usuario consultado.

def recomendar_peliculas_collaborative(user_id, ratings_matrix, user_similarity_df, top_n=8):
    # Obtener las películas que el usuario ya ha calificado
    rated_movies = ratings_matrix.loc[user_id].dropna().index

    # Obtener usuarios más similares
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).drop(user_id)

    # Tomar los usuarios más similares
    most_similar_users = similar_users.head(top_n).index

    # Recomendaciones basadas en los usuarios más similares
    recommended_movies = []
    for user in most_similar_users:
        movies_rated_by_similar_user = ratings_matrix.loc[user].dropna().index
        for movie in movies_rated_by_similar_user:
            if movie not in rated_movies:
                recommended_movies.append(movie)

    # Obtener los títulos de las películas recomendadas
    recommended_movie_titles = peliculas_df.loc[recommended_movies, 'titulo'].dropna()

    print(f"\n🎥 Películas recomendadas para el usuario {user_id}:\n")
    for i, movie in enumerate(recommended_movie_titles.head(top_n), 1):
        print(f"{i}. {movie}")


In [ ]:
#Ejemplo con el usuario 414
recomendar_peliculas_collaborative(414, ratings_matrix, user_similarity_df)


🎥 Películas recomendadas para el usuario 414:

1. Ace Ventura: When Nature Calls
2. From Dusk Till Dawn
3. Robin Hood: Men in Tights
4. Dead Man
5. Psycho
6. Seventh Seal, The (Sjunde inseglet, Det)
7. Carrie
8. Rosemary's Baby


#### SISTEMA DE RECOMENDACIÓN BASADA EN EL PERFIL DEL USUARIO.
---



```
# Para la recomendación basada en el perfil del usuario, se generó una matriz
 de calificaciones usuario-película. Los valores faltantes se completaron
con los promedios de calificación de cada usuario. Luego, se midió la similitud
 entre usuarios.

Con base en esta similitud, se recomendaron películas no vistas por el usuario
actual pero populares entre usuarios con perfiles similares,
empleando herramientas de análisis de matrices en Python.
```




In [ ]:
#La consulta permite filtrar las películas que ha visto ese usuario.

user_id = 5  # Se puede uedes cambiar este ID por el que necesites

query = """
SELECT m.*
FROM ratings r
JOIN movies m ON r.movieId = m.movieId
WHERE r.userId = ? AND r.rating >= 4.0
"""
pelis_usuario = pd.read_sql_query(query, conexion, params=(user_id,))

In [ ]:
#Permite evaluar de todos los generos el más frecuente que ha calificado.
generos = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
           'Documentary', 'Drama', 'Fantasy', "Film-Noir", 'Horror', 'IMAX',
           'Musical', 'Mystery', 'Romance', "Sci-Fi", 'Thriller', 'War', 'Western']

perfil_usuario = pelis_usuario[generos].sum().sort_values(ascending=False)
display(perfil_usuario.head(5))  # Mostrar los géneros más frecuentes

,0
Drama,14
Children,7
Crime,7
Comedy,7
Fantasy,5


In [ ]:
#Saca la consulta de las películas que no ha calificado
query = """
SELECT *
FROM movies
WHERE movieId NOT IN (
    SELECT movieId FROM ratings WHERE userId = ?
)
"""
pelis_no_vistas = pd.read_sql_query(query, conexion, params=(user_id,))

In [ ]:
# Multiplicamos cada fila por el perfil del usuario (ponderación por preferencia)
pelis_no_vistas['score'] = pelis_no_vistas[generos].dot(perfil_usuario)

# Ordenamos según el score
recomendaciones = pelis_no_vistas.sort_values(by='score', ascending=False)

# Mostramos las top 10 recomendaciones
display(recomendaciones[['movieId', 'titulo']].head(7))

#Ejemplo con el usuario 5


,movieId,titulo
3416,4719,Osmosis Jones
1346,1907,Mulan
8849,134853,Inside Out
7395,81132,Rubber
5655,27790,Millions
8303,108540,Ernest & Célestine (Ernest et Célestine)
1540,2123,All Dogs Go to Heaven


#### XAI PARA EL SISTEMA DE RECOMENDACION BASADO EN CONTENIDO.
---



```
# Objetivo:
Agregar una capa de explicación a las recomendaciones, para que el usuario entienda por qué se recomienda cada película.

Enfoque XAI para tu recomendador:
Tu recomendador usa similitud del coseno entre géneros.
Podemos explicar cuáles géneros compartidos son responsables
de que dos películas sean similares.


```



In [ ]:
#esta función toma los IDs de dos películas y el DataFrame de películas, identifica los géneros que ambas
#películas tienen en común y devuelve una lista con los nombres de esos géneros compartidos.

def explicar_recomendacion(pelicula_id_base, pelicula_id_recomendada, peliculas_df):
    # Extraer vectores de géneros (0s y 1s)
    base = peliculas_df.loc[pelicula_id_base].drop('titulo')
    recomendada = peliculas_df.loc[pelicula_id_recomendada].drop('titulo')

    # Comparar géneros (element-wise AND)
    generos_comunes = (base & recomendada)

    # Filtrar solo los géneros en común
    generos_activos = generos_comunes[generos_comunes == 1].index.tolist()

    return generos_activos

In [ ]:
#oma el título de una película, busca películas similares basándose en una matriz de similitud precalculada,
#selecciona las N películas más similares y luego imprime estas recomendaciones junto con los géneros que
#comparten con la película de entrada, proporcionando así una explicación básica de la recomendación.

def recomendar_peliculas_xai(movie_title, peliculas_df, similitud_df, top_n=4):
    # Buscar ID
    pelicula_id = peliculas_df[peliculas_df['titulo'].str.lower() == movie_title.lower()].index

    if pelicula_id.empty:
        print("Película no encontrada.")
        return

    pelicula_id = pelicula_id[0]

    # Calcular similitudes
    similares = similitud_df[pelicula_id].sort_values(ascending=False).drop(pelicula_id)

    # Seleccionar las top N
    recomendadas = similares.head(top_n)

    print(f"\n🎬 Películas similares a: {peliculas_df.loc[pelicula_id, 'titulo']}\n")

    for i, (rec_id, score) in enumerate(recomendadas.items(), 1):
        titulo = peliculas_df.loc[rec_id, 'titulo']
        generos_comunes = explicar_recomendacion(pelicula_id, rec_id, peliculas_df)
        print(f"{i}. {titulo} (Similitud: {score:.2f})")
        print(f"   ➤ Géneros en común: {', '.join(generos_comunes) if generos_comunes else 'Ninguno'}\n")


In [ ]:
# Ejemplo con la película de Forrest Gump
recomendar_peliculas_xai("Forrest Gump", peliculas_df, similitud_df)


🎬 Películas similares a: Forrest Gump

1. Life Is Beautiful (La Vita è bella) (Similitud: 1.00)
   ➤ Géneros en común: Comedy, Drama, Romance, War

2. I Served the King of England (Obsluhoval jsem anglického krále) (Similitud: 1.00)
   ➤ Géneros en común: Comedy, Drama, Romance, War

3. Train of Life (Train de vie) (Similitud: 1.00)
   ➤ Géneros en común: Comedy, Drama, Romance, War

4. Tiger and the Snow, The (La tigre e la neve) (Similitud: 1.00)
   ➤ Géneros en común: Comedy, Drama, Romance, War





```
# De acuerdo al resultado, podemos ver que las recomendaciones están basadas
en la similitud de los géneros y un resultado del 100% de similitud.

Basado en eso, realiza la recomendación el modelo.
```



#### FLUJO DE TRABAJO AUTOMATIZADO CON PIPELINES PARA EL SISTEMA DE RECOMENDACIÓN BASADO EN CONTENIDO.
------





```
# Usar los géneros de las películas para construir un perfil de usuario
y recomendarle películas similares a las que ha valorado alto (rating >= 4.0).
```





##### 1. CARGAR LA BASE DE DATOS USANDO LA CONEXIÓN CON SQL

In [ ]:
# Conectar a la base de datos SQLite
conexion = sql.connect('/content/drive/MyDrive/marketing/Marketing/data/db_movies')

# Cargar las tablas de la base de datos
movies = pd.read_sql_query("SELECT * FROM movies", conexion)
ratings = pd.read_sql_query("SELECT * FROM ratings", conexion)

##### 2. PREPROCERSAMIENTO

In [ ]:
# El GenreTransformer se encarga de transformar los géneros de las películas a una matriz binaria.
class GenreTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # Verificamos si los géneros están en cadenas separadas por '|', y si es así, los convertimos en listas
        if isinstance(X['genres'].iloc[0], str):
            X['genres'] = X['genres'].apply(lambda x: x.split('|'))

        # Entrenamos el MultiLabelBinarizer con las listas de géneros
        self.mlb = MultiLabelBinarizer()
        self.mlb.fit(X['genres'])
        return self

    def transform(self, X):
        # Nos aseguramos de que los géneros estén en formato de lista antes de transformarlos
        if isinstance(X['genres'].iloc[0], str):
            X['genres'] = X['genres'].apply(lambda x: x.split('|'))

        # Transformamos los géneros a una matriz binaria
        genre_matrix = pd.DataFrame(self.mlb.transform(X['genres']), columns=self.mlb.classes_, index=X.index)

        # Devolvemos la tabla original de películas junto con la matriz de géneros binarizados
        return pd.concat([X[['movieId', 'title']], genre_matrix], axis=1)


##### 3.PIPELINE

In [ ]:
# Crear el pipeline para procesar los géneros de las películas
genre_pipeline = Pipeline([
    ('genre_transformer', GenreTransformer())
])

# Aplicamos el pipeline a las películas
movie_features = genre_pipeline.fit_transform(movies)

##### 4. PERFIL DEL USUARIO


In [ ]:
def build_user_profile(user_id, min_rating=4.0):
    # Filtramos las películas calificadas alto por el usuario
    user_ratings = ratings[(ratings['userId'] == user_id) & (ratings['rating'] >= min_rating)]

    # Unimos las calificaciones con las características de las películas (géneros)
    user_movies = pd.merge(user_ratings, movie_features, on='movieId')

    # Promediamos los géneros de las películas favoritas
    genre_profile = user_movies[genre_cols].mean().values.reshape(1, -1)

    # Normalizamos el perfil
    return normalize(genre_profile)

##### 5. PIPELINE DE LA RECOMENDACIÓN

In [ ]:
#El pipeline de recomendación incluye tanto la creación del perfil del usuario como la recomendación de películas basadas en similitud.
class Recommender(BaseEstimator, TransformerMixin):
    def __init__(self, user_id, min_rating=4.0, top_n=10):
        self.user_id = user_id
        self.min_rating = min_rating
        self.top_n = top_n

    def fit(self, X, y=None):
        # Aquí no hacemos nada en fit, ya que solo necesitamos la data para el perfil del usuario
        return self

    def transform(self, X):
        # Generamos el perfil del usuario
        user_profile = build_user_profile(self.user_id, self.min_rating)

        # Películas vistas por el usuario
        seen_movies = ratings[ratings['userId'] == self.user_id]['movieId'].tolist()

        # Películas no vistas por el usuario
        unseen_movies = movie_features[~movie_features['movieId'].isin(seen_movies)].copy()

        # Calculamos las similitudes entre el perfil del usuario y las películas no vistas
        unseen_genres = unseen_movies[genre_cols].values
        similarities = cosine_similarity(user_profile, normalize(unseen_genres))[0]

        # Añadimos la columna de similitud
        unseen_movies['similarity'] = similarities

        # Seleccionamos las mejores recomendaciones
        top_recommendations = unseen_movies.sort_values(by='similarity', ascending=False).head(self.top_n)

        return top_recommendations[['title', 'similarity']]


##### 6. USO DEL SISTEMA USANDO PIPELINES

In [ ]:
# Crear el pipeline final para recomendaciones
recommender_pipeline = Pipeline([
    ('recommender', Recommender(user_id=5, min_rating=4.0, top_n=5))
])

# Ejecutar el pipeline de recomendaciones
recommendations = recommender_pipeline.transform(movies)

# Mostrar las recomendaciones
print("🎬 Recomendaciones para el usuario 5:")
print(recommendations)

🎬 Recomendaciones para el usuario 5:
                             title  similarity
5700               Millions (2004)    0.826898
1584  All Dogs Go to Heaven (1989)    0.785553
8900             Inside Out (2015)    0.773723
3460          Osmosis Jones (2001)    0.768742
7293        Leaves of Grass (2009)    0.762713


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
